In [1]:
pip install patool

In [2]:
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
import patoolib
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional



In [3]:
# Load Kaggle credentials and dataset
kaggle_config = json.load(open("kaggle.json"))
os.environ["KAGGLE_USERNAME"] = kaggle_config["username"]
os.environ["KAGGLE_KEY"] = kaggle_config["key"]
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

dataset = pd.read_csv("IMDB Dataset.csv")

# Preprocess data and split into training and testing sets
dataset.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)
train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

# Create tokenizer and fit to training data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_set["review"])

'kaggle' is not recognized as an internal or external command,
operable program or batch file.
C:\Users\Hp\AppData\Local\Temp\ipykernel_14272\2946503675.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [4]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_set["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_set["review"]), maxlen=200)
Y_train = train_set["sentiment"]
Y_test = test_set["sentiment"]


In [5]:

# Define LSTM model architecture
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=5000, output_dim=128, input_length=100))
lstm_model.add(LSTM(16, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1, activation="sigmoid"))


# Compile and train models
lstm_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


lstm_model.fit(X_train, Y_train, epochs=10, batch_size=128, validation_split=0.2)



C:\Users\Hp\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 50s 173ms/step - accuracy: 0.6807 - loss: 0.5955 - val_accuracy: 0.8395 - val_loss: 0.3801
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 41s 162ms/step - accuracy: 0.8422 - loss: 0.3794 - val_accuracy: 0.8407 - val_loss: 0.3608
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 41s 165ms/step - accuracy: 0.8694 - loss: 0.3280 - val_accuracy: 0.8446 - val_loss: 0.3545
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 40s 162ms/step - accuracy: 0.8770 - loss: 0.3022 - val_accuracy: 0.8510 - val_loss: 0.3477
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 40s 162ms/step - accuracy: 0.8888 - loss: 0.2808 - val_accuracy: 0.8586 - val_loss: 0.3438
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 40s 160ms/step - accuracy: 0.8932 - loss: 0.2698 - val_accuracy: 0.8591 - val_loss: 0.3443
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 40s 159ms/step - accuracy: 0.9047 - loss: 0.2477 - val_accuracy: 0.8665 - val_loss: 0.3416
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 40s 161ms/step - accuracy: 0.8983 - loss: 0

In [17]:
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test, Y_test)


print(f"LSTM Model - Test Loss: {lstm_loss:.4f}")
print(f"LSTM Model - Test Accuracy: {lstm_accuracy:.4f}")

# Define function to predict sentiment using LSTM model
def analyze_sentiment_lstm(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = lstm_model.predict(padded_sequence)
    return "positive" if prediction[0][0] > 0.5 else "negative"



313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - accuracy: 0.8721 - loss: 0.3418
LSTM Model - Test Loss: 0.3500
LSTM Model - Test Accuracy: 0.8675


In [19]:

# Example usage
new_review = "This movie was bad. I hate it."
lstm_sentiment = analyze_sentiment_lstm(new_review)
print(f"LSTM Model - The sentiment of the review is: {lstm_sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
LSTM Model - The sentiment of the review is: negative
